# Setup and Imports

Import useful libraries for defining our model, set up the model

In [1]:
from rdflib import Namespace, Graph
from buildingmotif import BuildingMOTIF
from buildingmotif.namespaces import BRICK
from buildingmotif.dataclasses import Model, Library

In [2]:
bm = BuildingMOTIF("sqlite://") # in-memory

In [3]:
BLDG = Namespace("urn:my_site/")
model = Model.create(BLDG)

In [4]:
qudtqk = Library.load(ontology_graph="http://qudt.org/2.1/vocab/quantitykind")
qudtunit = Library.load(ontology_graph="http://qudt.org/2.1/vocab/unit")
constraints = Library.load(ontology_graph="constraints/constraints.ttl")
brick = Library.load(ontology_graph="../libraries/brick/Brick.ttl")
g36 = Library.load(directory="../libraries/ashrae/guideline36")

/Users/gabe/Library/Caches/pypoetry/virtualenvs/buildingmotif-ozGZrh9J-py3.11/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


2024-07-02 17:36:18,082 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/facade/qudt (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:36:18,082 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/dimensionvector (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:36:21,884 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/prefix (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:36:21,885 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/sou (No row was found when one was required). Check this is loaded into BuildingMOTIF


...

2024-07-02 17:36:57,520 | root |  WARNING: Warning: could not find dependee n32fd143cc8f14ed985818aab5dd6561cb7632 in libraries []


2024-07-02 17:36:57,523 | root |  WARNING: Warning: could not find dependee n32fd143cc8f14ed985818aab5dd6561cb7638 in libraries []


2024-07-02 17:36:57,526 | root |  WARNING: Warning: could not find dependee n32fd143cc8f14ed985818aab5dd6561cb7634 in libraries []


2024-07-02 17:36:57,529 | root |  WARNING: Warning: could not find dependee n32fd143cc8f14ed985818aab5dd6561cb7636 in libraries []


2024-07-02 17:36:57,532 | root |  WARNING: Warning: could not find dependee n32fd143cc8f14ed985818aab5dd6561cb7630 in libraries []


- simulate looking at a mechanical schedule:
    - see number and type of equipment
    - **our example**: 1 AHU, 2 RVAVs
- now look at libraries to see if there are some useful "shapes"
    - based on forecasted app requirements
    - use these to inform model creation

In [5]:
sc = g36.get_shape_collection()

In [6]:
sc.get_shapes_about_class(BRICK.AHU)

[rdflib.term.URIRef('urn:ashrae/g36/4.8/sz-vav-ahu/sz-vav-ahu'),
 rdflib.term.URIRef('urn:ashrae/g36/4.6/mz-vav-ahu/AHU'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/fc-2'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/fc-4'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/fc-5'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/fc-6'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/fc-7'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/fc-8'),
 rdflib.term.URIRef('urn:ashrae/g36/5.16/multiple-zone-vav-air-handling-unit/outside_air_fraction')]

In [7]:
sc.get_shapes_about_class(BRICK.RVAV)

[rdflib.term.URIRef('urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat')]

# Write site manifest

- write a "manifest" to encode our requirements:
    - 1 AHU
    - 2 RVAVs; should run G36 sequence of operations
- this is the *low level* view -- will be abstracted by us and other developers

In [8]:
%%bash
cat <<EOF > my_site.ttl
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix constraint: <https://nrel.gov/BuildingMOTIF/constraints#> .
@prefix : <urn:my_site_constraints/> .

: a owl:Ontology ;
    owl:imports <https://brickschema.org/schema/1.4/Brick> ;
    owl:imports <https://nrel.gov/BuildingMOTIF/constraints> ;
    owl:imports <urn:ashrae/g36> .

# read off of mechanical schedule
:vav-count a sh:NodeShape ;
    sh:message "Need 2 VAVs" ;
    sh:targetNode : ; # target *this* graph
    constraint:exactCount 2 ;
    constraint:class <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .

:ahu-count a sh:NodeShape ;
    sh:message "Need 1 AHU" ;
    sh:targetNode : ; # target *this* graph
    constraint:exactCount 1 ;
    constraint:class brick:AHU .
EOF

Now we load our manifest into BuildingMOTIF

In [9]:
manifest = Library.load(ontology_graph="my_site.ttl")

2024-07-02 17:37:04,773 | root |  WARNING: An ontology could not resolve a dependency on urn:ashrae/g36 (No row was found when one was required). Check this is loaded into BuildingMOTIF


# Iterative model creation

- validate model to determine if it is *semantically sufficient*
- if this fails, BuildingMOTIF will use resulting report to help you patch the model

In [10]:
# pass our collections of shapes in to validate the model
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), 
                      g36.get_shape_collection(), brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Model is valid? {ctx.valid}")
# higher-level descriptions of each of the violations
print("Reasons why:")
for focus_node, diffs in ctx.diffset.items():
    print(focus_node)
    for diff in diffs:
        print("  - " + diff.reason())

2024-07-02 17:37:05,933 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:37:05,941 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries. Trying shape collections


2024-07-02 17:37:05,942 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:37:05,950 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries. Trying shape collections


2024-07-02 17:37:05,950 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/facade/qudt from Libraries (No row was found when one was required). Trying shape collections


...

2024-07-02 17:37:29,068 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/bacnet/2020 from Libraries. Trying shape collections


2024-07-02 17:37:29,068 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:37:29,081 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries. Trying shape collections


Model is valid? False
Reasons why:


None
  - Graph did not have 2 instances of urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat
  - Graph did not have 1 instances of https://brickschema.org/schema/Brick#AHU


- some constraints can be interpreted by BuildingMOTIF
- BuildingMOTIF identifies common patterns
    - presents these as new templates to the user
    - templates provide structure to input from user/other software

In [ ]:
generated_templates = ctx.as_templates()
print(generated_templates)
for t in generated_templates:
    print('-'*80)
    print(t.body.serialize())
    ident = input("Give value for 'name' in the above template: ")
    model.add_graph(t.evaluate({"name": BLDG[ident]}))

In [11]:
# now we can look at our model ( so far )
print(model.graph.serialize())

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:my_site/> a owl:Ontology .




In [12]:
# validate it again!
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), g36.get_shape_collection(), brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Model is valid? {ctx.valid}")
print("Reasons why:")
for focus_node, diffs in ctx.diffset.items():
    print(focus_node)
    for diff in diffs:
        print("  - " + diff.reason())

2024-07-02 17:37:53,702 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:37:53,719 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries. Trying shape collections


2024-07-02 17:37:53,720 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:37:53,736 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries. Trying shape collections


2024-07-02 17:37:53,737 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/facade/qudt from Libraries (No row was found when one was required). Trying shape collections


...

2024-07-02 17:38:23,858 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/bacnet/2020 from Libraries. Trying shape collections


2024-07-02 17:38:23,858 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:38:23,879 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries. Trying shape collections


Model is valid? False
Reasons why:


None
  - Graph did not have 1 instances of https://brickschema.org/schema/Brick#AHU
  - Graph did not have 2 instances of urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat


- model is still failing, so we ask BuildingMOTIF for more feedback
- ask user for more inputs on some of the missing sensors

- BuildingMOTIF can add missing metadata automatically
    - if unambiguous (true for some constraints)
    - otherwise, attempt "autocomplete" from existing model

In [13]:
print(model.graph.serialize())

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:my_site/> a owl:Ontology .




In [14]:
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), g36.get_shape_collection(), brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Is model valid yet? {ctx.valid}")

2024-07-02 17:38:51,962 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:38:51,987 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries. Trying shape collections


2024-07-02 17:38:51,988 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:38:52,011 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries. Trying shape collections


2024-07-02 17:38:52,012 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/facade/qudt from Libraries (No row was found when one was required). Trying shape collections


...

2024-07-02 17:39:23,007 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/bacnet/2020 from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:39:23,036 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/bacnet/2020 from Libraries. Trying shape collections


2024-07-02 17:39:23,037 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:39:23,066 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries. Trying shape collections


Is model valid yet? False


In [15]:
print(ctx.report.serialize())

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <urn:my_site_constraints/> ;
            sh:resultMessage "Need 2 VAVs",
                "Not the right number 2 of instances of urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent <https://nrel.gov/BuildingMOTIF/constraints#countConstraintComponent> ;
            sh:sourceShape <urn:my_site_constraints/vav-count> ;
            sh:value <urn:my_site_constraints/> ],
        [ a sh:ValidationResult ;
            sh:focusNode <urn:my_site_constraints/> ;
            sh:resultMessage "Need 1 AHU",
                "Not the right number 1 of instances of https://brickschema.org/schema/Brick#AHU" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent <https://nrel.

 - we can also invoke templates ourselves, rather than having BuildingMOTIF suggest them
 - here using `fill()` as a debug trick to avoid having to give inputs

In [16]:
# use templates to finish it up!
vav_templ = g36.get_template_by_name("vav-terminal-unit-reheat").inline_dependencies()
_, graph = vav_templ.evaluate({"name": BLDG["vav1"]}).fill(BLDG)
model.add_graph(graph)
_, graph = vav_templ.evaluate({"name": BLDG["vav2"]}).fill(BLDG)
model.add_graph(graph)

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "name-dmp, rhc-cmd, rhc, name-dmp-dmppos, name-co2, name-zone, name-ztemp, name-dat, name-occ" were not provided during evaluation
  warnings.warn(


In [17]:
ctx = model.validate([manifest.get_shape_collection(),
                      constraints.get_shape_collection(),
                      g36.get_shape_collection(),
                      brick.get_shape_collection()], error_on_missing_imports=False)
print(f"Is model valid yet? {ctx.valid}")

2024-07-02 17:39:50,383 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:39:50,416 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of https://brickschema.org/schema/Brick/ref from Libraries. Trying shape collections


2024-07-02 17:39:50,417 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:39:50,449 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/shacl/overlay/qudt from Libraries. Trying shape collections


2024-07-02 17:39:50,450 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/schema/facade/qudt from Libraries (No row was found when one was required). Trying shape collections


...

2024-07-02 17:40:23,820 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/bacnet/2020 from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:40:23,858 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/bacnet/2020 from Libraries. Trying shape collections


2024-07-02 17:40:23,859 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:40:23,896 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://qudt.org/2.1/collection/usertest from Libraries. Trying shape collections


Is model valid yet? False


In [18]:
print(model.graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:my_site/> a owl:Ontology .

<urn:my_site/vav1> a brick:VAV ;
    brick:feeds <urn:my_site/name-zone_130e143f> ;
    brick:hasPart <urn:my_site/name-dmp_d98bd58e>,
        <urn:my_site/rhc_2c8a0aba> ;
    brick:hasPoint <urn:my_site/name-dat_33c6c751>,
        <urn:my_site/name-ztemp_62c77ab7> .

<urn:my_site/vav2> a brick:VAV ;
    brick:feeds <urn:my_site/name-zone_cbfab8b2> ;
    brick:hasPart <urn:my_site/name-dmp_87ea638c>,
        <urn:my_site/rhc_d0e031ee> ;
    brick:hasPoint <urn:my_site/name-dat_723731f5>,
        <urn:my_site/name-ztemp_fa98c50f> .

<urn:my_site/name-dat_33c6c751> a brick:Discharge_Air_Temperature_Sensor .

<urn:my_site/name-dat_723731f5> a brick:Discharge_Air_Temperature_Sensor .

<urn:my_site/name-dmp-dmppos_4190eb9e> a brick:Damper_Position_Command .

<urn:my_site/name-dmp-dmppos_e4423605> a brick:Damper_Position_Command .

<urn:my_site/name-dmp_8

In [19]:
print(ctx.report.serialize())

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <urn:my_site/name-zone_130e143f> ;
            sh:resultMessage "urn:my_site/name-zone_130e143f has type https://brickschema.org/schema/Brick#HVAC_Zone which was deprecated in version 1.4.0. For now, it has been updated to also be of type https://w3id.org/rec#HVACZone." ;
            sh:resultSeverity sh:Warning ;
            sh:sourceConstraint _:n32fd143cc8f14ed985818aab5dd6561cb3688 ;
            sh:sourceConstraintComponent sh:SPARQLConstraintComponent ;
            sh:sourceShape <https://brickschema.org/schema/BrickShape#DeprecationRuleForInstances> ;
            sh:value <urn:my_site/name-zone_130e143f> ],
        [ a sh:ValidationResult ;
            sh:focusNode <urn:my_site_constraints/> ;
            sh:resultMessage "Need 2 VAVs",
                "Not the right numb